## Validación de datos

Vamos a hacer ahora una clase práctica de navegacion en los datos, compararemos dos dfs donde uno es un espejo del otro con errorcillos para intentar encontrar esos errores, esas líneas diferentes que luego nos conducirian en el código a buscar la sentencia corrupta

#### Importamos las librerías necesarias para hacer el análisis

#### Cargamos el dataset

Crea dos dataframes, uno df_test y el otro df_output. El primero debe leer la hoja test del excel Validación_datos_example y el segundo la hoja Output del mismo excel

#### Asegurarse de que test y output tienen los mismos formatos

In [1]:
# Empezamos viendo las primeras filas de los dfs para ver a simple vista si tienen el mismo aspecto


#### Analizar las columnas categóricas por valores únicos en test vs output

In [2]:
# miramos todos los valores únicos de las columnas categóricas
# Lo primero es saber cuales son las cátegoricas, podemos verlo a simple vista , o filtrar el df por el tipo de columna 


In [3]:
# como son iguales las columnas, ahora recorremos los diccionarios para ver las desigualdades dentro de cada columna
# imprimimos el nombre de la columna y los valores diferentes de esa columna entre el df_output y el df_test


Vemos que las diferencias son los valores nan, eso es porque en python nan != nan, por lo que podriamos tratar los nans y luego repetir, o sencillamente ya sabemos que no hay valores unicos diferentes entre las columnas categóricas de ambos dfs

In [4]:
# Comprobamos si el tipo de datos de las columnas es igual


A simple vista parece el mismo df

#### Analizar las columnas numéricas en test vs output

In [5]:
# Comprobamos tambien si el tamaño de los dfs y los nulos son iguales


Hay que fijarse pero vemos que hay menos nulls en el numero de puertas de la salida que en el de test

In [6]:
# Comprobamos si los valores numéricos son iguales mirando sus variables estadísticas
# Si todos los valores son iguales, es probable que la salida de nuestro proceso sea correcta y dé lo que esperábamos


En el precio bailan la media y en la desviación estándar

In [8]:
# Vamos a ver la media que es algo que no muestra el describe de serie


Algo no está bien, nos toca buscar que es, vamos a entrar en los datos poco a poco

#### Comparamos los dataframes para buscar registros difrentes en ellos, de esa manera si son pocos sacamos las diferencias

In [9]:
# Para eso los mergeamos marcando las lineas con el indicador de si esa fila está en los dos, en la izquierda o en la derecha


In [10]:
# Y las unimos en un solo df para verlas juntas y ordenadas por las diferencias


Otra manera más rápida seria unir los dos dfs y quitar las lineas duplicadas, asi sólo nos quedarian las diferentes

Este segundo método tendria el ligerísimo inconveniente de que no sabes que registro viene de cada lado

Ahora nos tocaría filtrar las diferencias para aislar un caso determinado, luego buscar en el código dónde se crea esa línea y por último, seguirla para ver dónde se realizar la operación o cálculo erróneo.

Sea como fuere, el procedimiento para encontrar las diferencias y aislarlas es siempre el que hemos visto, iterando hasta que el resultado "output" sea justo lo que esperabamos , tengamos un test con el que compararnos o no.

Tambien existe una aproximación diferente, donde se agrupan los valores y se va profundizando progresivamente para ver buscar la línea exacta que esta mal.

Esto es más lento que comparar los dos dfs como hemos hecho antes pero por el camino, al ir paso a paso, muchas veces detectamos problemas que se explican mejor con los valores agrupados que linea a linea. 

Por ejemplo si se realizan calculos agrupados por marca o por modelo , la linea quizás no nos refleje todo el problema.

Vamos a ver ahora cómo hilar muy fino, yendo de menos detalle a más para buscar valores diferentes entre los dos dataframes artesanalmente

#### Agrupamos datos y buscamos los problemas

De mayor a menor: Se agrupan los datos por los conjuntos más grandes, si no son iguales, se va aumentando la granularidad hasta el máximo posible en el test y el output

Nos quedamos con el precio para investigar más, y de categórica elegimos en primera instancia la Marca

Juntamos los dfs agrupados para buscar las marcas que no son iguales en ambos lados

Ya sabemos que dos marcas son distintas, ahora repetimos el proceso añadiendo otra columna al group by pero filtrando por Kia

Ya sabemos que el KIA Stonic nos está devolviendo diferencias, vamos a ver que lineas son esas en los dfs, cuantas hay y con esto podriamos entender la lógica del error en nuestro código para ver porqué esa diferencia existe

Todavia hay muchos registros asi que tendríamos que seguir iterando sobre otras columnas como el año , el tipo de combustible, etc... hasta encontrar los registros que nos están haciendo el lío y despues entender por qué nos lo están haciendo...

#### ¡Eureka!

No sólo hemos encontrado la diferencia, sino que además es algo que NO habiamos visto de la otra manera, y es que si nos fijamos, hay una línea distinta, la 38702, pero es que además esa línea habia quedado enmascarada con el procedimiento de buscar las diferencias en el df completo por estar duplicada en el df de test, de forma que si existina en ambos dfs, solo que en el de la salida estaba una sóla vez.

## Ejercicio

Comparar la hoja Test vs hoja Exercise para buscar las diferencias, sabemos que nuestro proceso saca valores que no estan siempre bien pero no sabemos porqué, ni cuale son, encuentra porqué y despues encuentra la lógica del error.

Ojo!, porque aqui ya vemos cosas raras, como que todos los que han cambiado tienen Tipo de combustible Híbridos Tipo de cambio manual.

 Nos da una pista, sin embargo aun se nos escapa algo porque ese filtro nos devovlería más lineas que las diferentes y además no entendemos todavia bien qué es lo que hace mal.
 
 Vamos a coger y buscar relacion entre las lineas modificadas y las originales para intentar encontrar que pasa en el cálculo

A veces existe una relation2 de 0,5 y a veces no, eso nos indica que los valores se están duplicando, pero además nos dice que no siempre. 

Esto está mal, sabrias decir porque no siempre es 0,5?

Efectivamente!!, el problema es que hemos cruzado por todos los campos peero resulta que tenemos varios registros iguales con la misma conbinacion de campos , por lo que el cruce nos está devolviendo valores que no existen realmente, se produce un producto cartesiano parcial sin querer

Vamos a hacer otra vez la diferencia pero sobre los campos agrupados para ver el total vs total

#### ¡Eureka!

Todas las relaciones2 son 0,5, ahora sí!!!, ya hemos descubierto que estaba sucediendo, y es que duplica los valores de algunos registros!.

Además, ya sabemos que esos registros son los hibridos manuales , pero aun os queda afinal algo más, proque son los hibridos manuales y algo más , vamos a ver si descubrimos que condiciones nos faltan por encontrar.

Cogemos un modelo del df original y buscamos los que se han modificado y los que no , es la diferencia entre los que si y los que no la que nos va a dar la pista final

Nos quedamos con los que son diferentes y buscamos la diferencia entre el lineas_diff_filtered vs lineas_original_filtered

Vemos que ni la marca, ni el modelo , ni el tipo de combustible, ni el año , ni el numero de puertas , ni el tipo de cambio nos estan afectando, eso solo nos deja el precio y los kilómetros. Y como el precio es lo que se ha modificado realmente, sólo puede ser algo que sucede en los kilómetros.

¿Sabrías ver que es?

#### Efectivamente!

Las lineas que se modifican tienen kms pequeños o dicho de otro modo, las que no se modifican tienen valores más altos en la columna kms, vamos a recuperar los valores unitarios de esa columna de las lineas que se habian modificado y que en su momento nos pasó desapercibido:

 'Kilómetros': array([ 350.,    6.,    1.,   10.,  300.,    0.,    5.,    7.,    2.,
        1000.,    9.,   15.,   25.,   30.,   38.,    3.,   12.,   50.,
         293.,   17.,    8.,    4.,   14.])

Vemos que todos los valores están entre 1 y 1000, será esta la condicion?, vamos a aplicarle el filtro al df original otra vez vs el filtrado a ver si llegamos al mismo numero de lineas, esa puede ser una comprobación fácil y rápida

#### ¡Eureka!

Ahora si hemos encontrado la respuesta a las dos primeras preguntas!, 

¿Qué está sucendiendo? --> Está duplicando el valor del precio 
¿Por qué está sucendiendo? --> Lo hace sólo en aquellas lineas de vehiculos manuales , hibridos y con menos de 1000 kms
¿Dónde está sucediendo? --> Ahora con esta información ya podemos debugar el código buscando donde se producen los cálculos de la columna precio para resolverlo!.

Es posible que el error sea de parametrización, que ya veremos en un proximo capítulo o por un filtro mal hecho, un cruce no deseado, una coma donde no debe... hay mil motivos, pero lo importante es que hayas entendido que para que un producto sea industrializable y funcional en una empresa, los valores deben ser validados a conciencia, y si falla en algún momento, debemos ser capaces de sumergirnos hasta el fondo para encontrar el origen del problema.

En nuestra experiencia profesional hemos visto decenas de programadores y data scientists que les gusta mucho programar pero luego no saben profundizar en los numeros para validar sus códigos... ERROR!, Un perfil completo debe ser capaz de resolverlo todo, de la A-z